In [44]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import pandas as pd
import argparse                 
import sklearn 
import numpy as np 
import logging 
import sklearn.metrics as metrics 
import scikitplot as skplt
import matplotlib.pyplot as plt                                  
from argparse import ArgumentParser
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import DataLoader, TensorDataset, Subset, Dataset, random_split
import torch.nn.functional as F
from collections import Counter, defaultdict
import itertools
import os 
import copy
import warnings
from dataload import makedata
from classifier import ClassifierTrainer
from models import VQ_Classifier
import itertools
from data import load_data
torch.manual_seed(911) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [46]:
batch_size = 1
len_position=12
dataset = 'mitbih'
classifier = 'cnn'
classification_model = "/home/smjo/xai_timeseries/vqvae/saved_models/classification/mitbih/8/cnn.pt"
vqvae_model = "/home/smjo/xai_timeseries/vqvae/saved_models/hard_mitbih/8/model_290.pt"
ds = load_data(dataset, task = 'classification')



test_size = 80
train_size = len(ds)-test_size
train_dataset,test_dataset = random_split(ds, [train_size,test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, pin_memory=True)

#Find masking token
end_tokens={}

net = VQ_Classifier(
    num_classes = 2,
    vqvae_model = vqvae_model,
    positions =0,
    mask = 0,
    auc_classification = False,
    model_type = classifier,
    len_position = len_position
).to(device)

a = torch.load(classification_model)
net.load_state_dict(a['model_state_dict'])

for param in net.parameters():
    param.requires_grad = False

task is classification
dataset is mitbih
X shape:torch.Size([8039, 192]), y shape:torch.Size([8039, 2])


/home/smjo/xai_timeseries/data.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  n_set = torch.tensor(torch.load('./mit_bih_dataset/n_data.pt'))
/home/smjo/xai_timeseries/data.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s_set = torch.tensor(torch.load('./mit_bih_dataset/s_data.pt'))
/home/smjo/xai_timeseries/data.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v_set = torch.tensor(torch.load('./mit_bih_dataset/v_data.pt'))
/home/smjo/xai_timeseries/data.py:125: UserWarning: To copy construct from a tensor, it is recomm

In [47]:
def compare(codebook_tokens,pos, item, quant):
    if codebook_tokens[0][pos][quant] == item[0][quant]:
        return 'exist'
    else:
        return 'not_exist'
    

### region : 몇번째 quantizer 까지 볼껀지 ###

In [66]:

count_code_class0 = {pos:list() for pos in range(len_position)}
count_code_class1 = {pos:list() for pos in range(len_position)}
class0, class1=0,0
region=3 
with torch.no_grad():    
    for _, (data, labels) in enumerate(test_loader):
        data = data.unsqueeze(1).float()
        labels = labels.type(torch.LongTensor)
        labels = torch.argmax(labels, dim=1)
        data, labels = data.to(device), labels.to(device)
        output, codebook_tokens, recon, input= net(data)
        codebook_tokens = codebook_tokens.cpu().detach().numpy()
        if labels==0:
            class0+=1
            for pos in range(len_position):
                
                state = 'not_exist'
                for item_idx,item in enumerate(count_code_class0[pos]):
                    for quant in range(region):
                        state = compare(codebook_tokens,pos, item,quant)
                        if state =='not_exist':
                            break
                    if state =='exist':
                        count_code_class0[pos][item_idx][1] +=1
                        break
                if state =='not_exist':
                    count_code_class0[pos].append([tuple(codebook_tokens[0][pos][:region]),1])
                    continue

        else:
            class1+=1
            for pos in range(len_position):
                state = 'not_exist'
                for item_idx,item in enumerate(count_code_class1[pos]):
                    for quant in range(region):
                        state = compare(codebook_tokens,pos, item,quant)
                        if state =='not_exist':
                            break
                    if state =='exist':
                        count_code_class1[pos][item_idx][1] +=1
                        break
                if state =='not_exist':
                    count_code_class1[pos].append([tuple(codebook_tokens[0][pos][:region]),1])
                    continue

In [67]:
count_overlapping = {pos:0 for pos in range(len_position)}
for pos in range(len_position):
    for i,code_i in enumerate(count_code_class0[pos]):
        for k, code_k in enumerate(count_code_class1[pos]):
            if code_i[0] == code_k[0]:
                #print(f"position:{pos}, codebook:{code_k[0]}, class0:{code_k[1]}, class1: {code_i[1]}")
                count_overlapping[pos]+=min(code_i[1], code_k[1])

#### class 간 position 별로 겹치는 sample 개수, class0 sample 개수, class 1 sample 개수 ###

In [68]:
count_overlapping , class0, class1

({0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 1, 6: 2, 7: 4, 8: 6, 9: 7, 10: 7, 11: 7},
 67,
 13)